# Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from helpers import parse_variables, get_risk_level, hi_gauss_blob_risk_fun, blob_risk_fun, NW_risk_fun, square_risk_fun, map_to_color
from matplotlib.colors import LinearSegmentedColormap
import importlib.util
from k_means_constrained import KMeansConstrained


import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from helpers import parse_variables, get_risk_level, map_to_color, simulate_quant_trait
from models import ols_regression, manhattan_linear, gc
from deep_learning_models import abyss, deep_abyss

from scipy.stats import t
from scipy import stats
from scipy.stats import entropy

from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras import Input, Model, layers, regularizers
from tensorflow.keras.layers import Input, Dense

2024-08-15 16:12:05.547753: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-15 16:12:05.597431: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-15 16:12:05.750384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 16:12:05.778375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 16:12:05.778410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 16:12:05.793153: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
k = int(dict['k'])
M = float(dict['M'])

# Thresholds
very_rare_threshold_L = float(dict['very_rare_threshold_L'])
very_rare_threshold_H = float(dict['very_rare_threshold_H'])

rare_threshold_L = float(dict['rare_threshold_L'])
rare_threshold_H = float(dict['rare_threshold_H'])

common_threshold_L = float(dict['common_threshold_L'])
common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k

In [3]:
very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/02_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/02_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/02_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

In [4]:
very_rare = very_rare.rename(columns=lambda x: 'VR' + x)/2
rare = rare.rename(columns=lambda x: 'R' + x)/2
common = common.rename(columns=lambda x: 'C' + x)/2
complete = pd.concat([common, rare, very_rare], axis=1)
complete = ((complete*2)-1)

# Divide in LD blocks

In [5]:
genos = complete.T

In [6]:
sample_size = 200
n_components = 15
size_min = sample_size - round(sample_size / 5)
size_max = sample_size + round(sample_size / 5)

In [7]:
# Calculate the number of clusters based on sample size
num_clus = round(genos.shape[0] / sample_size)

In [8]:
# Standardize the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(genos)

In [9]:
# Perform PCA with n components
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(df_scaled)

# Create a new DataFrame to store the principal components
pc_columns = [f'PC{i+1}' for i in range(n_components)]
df_pca = pd.DataFrame(data=principal_components, columns=pc_columns)

# Apply constrained K-Means clustering
clf = KMeansConstrained(
    n_clusters=num_clus,
    size_min=size_min,
    size_max=size_max,
    random_state=0
)
clf.fit_predict(np.array(df_pca))


array([3, 6, 6, ..., 0, 0, 7], dtype=int32)

In [10]:
genos['clusters_k_means'] = clf.labels_

In [11]:
PATH_output = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/LD_blocks"
os.system(f"rm -rf {PATH_output}")
os.makedirs(PATH_output, exist_ok=True)

In [12]:
for num in genos.clusters_k_means.unique():
    to_save = genos.loc[genos['clusters_k_means'] == num]
    to_save = to_save.drop(columns=['clusters_k_means']).T

    # Calculate minimum and maximum MAF for the cluster
    tot_mafs = []
    for snp_id in to_save:
        try:
            try:
                num_maj = to_save[[snp_id]].value_counts()[1]
            except Exception as e:
                num_maj = 0
            try:
                num_het = to_save[[snp_id]].value_counts()[0]
            except Exception as e:
                num_het = 0
            try:
                num_min = to_save[[snp_id]].value_counts()[-1]
            except Exception as e:
                num_min = 0
            total_humans = num_maj + num_het + num_min
            maf = (num_min*2 + num_het)/(total_humans*2)

        except Exception as e:
            print(e)
            print(f"snp {snp_id} has a problem")

        tot_mafs.append(maf)

    max_maf = np.round(max(tot_mafs), 5)
    min_maf = np.round(min(tot_mafs), 5)
    size = to_save.shape[1]

    # Save the processed cluster data
    to_save.to_pickle(f"{PATH_output}/{num}_{size}_maf_{min_maf}_{max_maf}.pkl")

# Create one hot encoded genotype for LD blocks

In [14]:
path_one_hot_genotype = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/LD_blocks_one_hot/"
os.system(f"rm -rf {path_one_hot_genotype}")
os.makedirs(path_one_hot_genotype, exist_ok = True)

In [15]:
path_lds = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/LD_blocks"
ld_files = os.listdir(path_lds)
for ld_file in ld_files:
    path_ld_file = path_lds + "/" + ld_file
    complete = pd.read_pickle(path_ld_file)

    # Create db_minor
    db_minor = complete.copy()
    db_minor = db_minor.applymap(lambda x: 1 if x == -1.0 else 0)
    
    # Create db_het
    db_het = complete.copy()
    db_het = db_het.applymap(lambda x: 1 if x == 0.0 else 0)
    
    # Create db_major
    db_major = complete.copy()
    db_major = db_major.applymap(lambda x: 1 if x == 1.0 else 0)
    

    db_minor.to_pickle(f"{path_one_hot_genotype}{ld_file.split('.pkl')[0]}_db_minor.pkl")
    db_het.to_pickle(f"{path_one_hot_genotype}{ld_file.split('.pkl')[0]}_db_het.pkl")
    db_major.to_pickle(f"{path_one_hot_genotype}{ld_file.split('.pkl')[0]}_db_major.pkl")